<a href="https://colab.research.google.com/github/yassine-fetoui/artificial-intelligence/blob/main/Good_Bad_sperm_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

In [ ]:
bad_sperm_count=len(os.listdir('/content/drive/MyDrive/High Quality Sperm - Labeled/Bad Sperm'))
good_sperm_count=len(os.listdir('/content/drive/MyDrive/High Quality Sperm - Labeled/Good Sperm'))

In [ ]:
data = pd.DataFrame({
    'Category': ['Bad Sperm', 'Good Sperm'],
    'Count': [bad_sperm_count, good_sperm_count]
})

In [ ]:
data

In [ ]:
sns.barplot(x=data['Category'], y=data['Count'] , hue=data['Count'])
plt.title("Count of images in each categoy")
plt.xlabel("Sperm Quality")
plt.ylabel("Number of Images")
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.pie(data['Count'],labels=data["Category"],autopct='%5.1f%%', startangle=45)
#plt.axis('equal')
plt.title("Count of images in each categoy")
plt.show()

In [ ]:
from PIL import Image

In [ ]:
def display_image_from_dir(path, num_image, title):
  images=os.listdir(path)[:num_image]
  plt.figure(figsize=(15,6))
  plt.suptitle(title,fontsize=16)
  for i , img in enumerate(images):
    imgf=os.path.join(path,img)
    imgo=Image.open(imgf)
    plt.subplot(1,num_image,i+1)
    plt.imshow(imgo)
    plt.axis('off')
    plt.title(f'Image {i+1}')
  plt.tight_layout()
  plt.show()


In [ ]:
display_image_from_dir('/content/drive/MyDrive/High Quality Sperm - Labeled/Bad Sperm', 5, 'Bad Sperm')
display_image_from_dir('/content/drive/MyDrive/High Quality Sperm - Labeled/Good Sperm', 5, 'Good Sperm')

In [ ]:
bad_sperm_image=[os.path.join('/content/drive/MyDrive/High Quality Sperm - Labeled/Bad Sperm', img )
for img in os.listdir('/content/drive/MyDrive/High Quality Sperm - Labeled/Bad Sperm')]
good_sperm_image=[os.path.join('/content/drive/MyDrive/High Quality Sperm - Labeled/Good Sperm', img )
for img in os.listdir('/content/drive/MyDrive/High Quality Sperm - Labeled/Good Sperm')]


bad_sperm_df=pd.DataFrame({'image':bad_sperm_image, 'label':'bad'})
good_sperm_df=pd.DataFrame({'image':good_sperm_image, 'label':'good'})

df=pd.concat([bad_sperm_df, good_sperm_df], axis=0,ignore_index=True) # 123/456 ignore

In [ ]:
df.tail(5)

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
from imblearn.over_sampling import RandomOverSampler


In [ ]:
df.describe()

In [ ]:
from re import X
ros=RandomOverSampler(random_state=42)
X_res,y_res=ros.fit_resample(df[['image']],df['label'])

In [ ]:
X_res.shape

In [ ]:
X_res

In [ ]:
y_res.describe()

In [ ]:
bala_train_df=pd.concat([X_res,y_res],axis=1)

In [ ]:
bala_train_df["image"][0]

In [ ]:
bala_train_df

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_df, temp_df= train_test_split(bala_train_df, test_size=0.2, random_state=42, stratify=bala_train_df['label'] )
val_df, test_df= train_test_split(temp_df, test_size=0.5, random_state=42 , stratify=temp_df['label'])


print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
image_size =(224,224)
batch_size=32

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_gen=train_datagen.flow_from_dataframe(
    train_df,
    x_col='image',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',

)

val_gen=val_test_datagen.flow_from_dataframe(
    val_df,
    x_col='image',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)


test_gen=val_test_datagen.flow_from_dataframe(
    test_df,
    x_col='image',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

In [ ]:
np.ceil(len(df)/32)

In [ ]:
from tensorflow.keras.utils import Sequence
import numpy as np
import cv2


class DataGenerator(Sequence):
  def __init__(self, df, batch_size, image_size, shuffle=True):
    self.df=df
    self.batch_size=batch_size
    self.image_size=image_size,
    self.shuffle=shuffle
    self.on_epoch_end()
  def __data_generation(self,batch) :
    X=np.empty((self.batch_size,*self.image_size,3))
    y=np.empty((self.batch_size),dtype=int)

    for i,path in enumerate(batch['image']):
      img=cv2.imread(path)
      img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      img=cv2.resize(img,self.image_size)
      X[i]=img/255.0
      y[i]=1 if batch.iloc[i]['label']=='good' else 0
    return X,y

  def __len__(self):
      return int(np.ceil(len(self.df)/self.batch_size))

  def __getitem__(self,index):
      self.indexs=index[self.index*batch_size : (index+1)*batch_size]
      batch=self.df.iloc[self.indexs]
      X,y=self.__data_generation(batch)
      return X,y
  def on_epoch_end(self):
      self.indexs=np.arange(len(self.df))
      if self.shuffle:
        np.random.shuffle(self.indexs)





In [ ]:
image_size=(255,255)
batch_size=32
train_generator = DataGenerator(train_df, batch_size=batch_size,
image_size=image_size)
validation_generator = DataGenerator(val_df, batch_size=batch_size,
image_size=image_size)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
def build_unet(input_size=(224, 224, 3)):
    inputs = layers.Input(input_size)

    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

input_shape = (224, 224, 3)
unet = build_unet(input_shape)

unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

In [ ]:
unet.summary()

In [ ]:
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

In [ ]:
history = unet.fit(
 train_generator,
 validation_data=validation_generator,
 epochs=5
)